In [ ]:
# AIRE Micro Tutor Identity Cell
import datetime as dt
learner_id = input('Enter your learner ID (email or handle): ').strip()
learner_role = input('Enter your learner role (e.g., researcher, educator): ').strip()
resource_id = 'notebook:rag_query'
session_id = f"{learner_id}-{dt.datetime.utcnow().isoformat()}"

In [ ]:
# AIRE Micro Tutor Imports
from prompt_tutor import evaluate_prompt
from aire_telemetry import log_event
from resources_map import RESOURCE_MAP

In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# Query the Retrieval Index

**What**: Retrieve relevant documents from the pre-built TF-IDF index using natural language queries.

**Why**: Retrieval is the core component of RAG (Retrieval-Augmented Generation) systems, allowing them to access external knowledge.

**How**:
1. **Load the saved index**.
2. **Transform a user query** into a vector.
3. **Calculate cosine similarity** to find the closest documents.

**Key Concept**: **Cosine Similarity** measures the cosine of the angle between two vectors. A value close to 1 means the vectors (and thus the texts) are very similar.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You loaded the TF-IDF index.
- You ran at least one query.
- You viewed a ranked list with scores.

In [ ]:
from pathlib import Path


def find_data_dir() -> Path:
    candidates = [Path.cwd() / "data", Path.cwd().parent / "data", Path.cwd().parent.parent / "data"]
    for candidate in candidates:
        if (candidate / "sample_texts" / "articles_sample.csv").exists():
            return candidate
    raise FileNotFoundError("data directory not found. Run scripts/generate_synthetic_data.py.")


DATA_DIR = find_data_dir()


In [ ]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

articles = pd.read_csv(DATA_DIR / "sample_texts" / "articles_sample.csv")
index_path = DATA_DIR / "vector_index.pkl"

if not index_path.exists():
    vectorizer = TfidfVectorizer(stop_words="english")
    tfidf_matrix = vectorizer.fit_transform(articles["abstract"].fillna(""))
    with open(index_path, "wb") as handle:
        pickle.dump({"vectorizer": vectorizer, "tfidf_matrix": tfidf_matrix}, handle)
else:
    with open(index_path, "rb") as handle:
        payload = pickle.load(handle)
    vectorizer = payload["vectorizer"]
    tfidf_matrix = payload["tfidf_matrix"]


query = "methods for reproducible ai"
query_vec = vectorizer.transform([query])
scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
articles = articles.assign(score=scores)
articles.sort_values("score", ascending=False).head(5)[["title", "score"]]


In [ ]:
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

articles = pd.read_csv(DATA_DIR / "sample_texts" / "articles_sample.csv")
with open(DATA_DIR / "vector_index.pkl", "rb") as handle:
    payload = pickle.load(handle)
vectorizer = payload["vectorizer"]
tfidf_matrix = payload["tfidf_matrix"]

query = "methods for reproducible ai"
query_vec = vectorizer.transform([query])
scores = cosine_similarity(query_vec, tfidf_matrix).flatten()
articles = articles.assign(score=scores)
articles.sort_values("score", ascending=False).head(5)[["title", "score"]]


### If you get stuck / What to try next

If you get stuck: rebuild the index or clear and rerun cells. What to try next: tune prompts in pipelines/rag/rag_evaluation.ipynb or integrate answers in pipelines/prototypes/minimal_research_assistant.ipynb.

In [ ]:
# AIRE Micro Tutor Prompt + Feedback Cell
from aire_llm_client import chat_completion

def call_llm(prompt_text: str) -> str:
    messages = [
        {'role': 'system', 'content': 'You are an AI helper responding concisely.'},
        {'role': 'user', 'content': prompt_text},
    ]
    return chat_completion(messages)

prompt_text = input('Enter the prompt you want to test: ').strip()
ai_response = call_llm(prompt_text)
evaluation = evaluate_prompt(prompt_text, learner_role)

clarity = evaluation.get('clarity_score')
context = evaluation.get('context_score')
constraints = evaluation.get('constraints_score')
evaluation_score = evaluation.get('evaluation_score')
primary_weakness = evaluation.get('primary_weakness', '')
recommended_resource_id = RESOURCE_MAP.get(primary_weakness) or RESOURCE_MAP.get('evaluation')

log_event(
    event_name='micro_tutor_evaluation',
    user_id=learner_id or session_id,
    metadata={
        'resource_id': resource_id,
        'learner_role': learner_role,
        'clarity': clarity,
        'context': context,
        'constraints': constraints,
        'evaluation_score': evaluation_score,
        'primary_weakness': primary_weakness,
    },
)

print('\n--- AI Response ---')
print(ai_response)
print('\n--- Feedback ---')
print(evaluation.get('summary', 'No summary provided.'))
print('Primary weakness:', primary_weakness or 'n/a')
print('Strengths:', ', '.join(evaluation.get('strengths', [])) or 'n/a')
print('Suggestions:', ', '.join(evaluation.get('suggestions', [])) or 'n/a')
print('\nRecommended resource:', recommended_resource_id or 'n/a')